In [1]:
# !python3 -m pip install --upgrade langchain deeplake openai tiktoken

In [1]:
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import DeepLake
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [2]:
import openai
import os
import IPython
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
ACTIVELOOP_TOKEN = os.getenv('ACTIVELOOP_TOKEN')
org = os.getenv('ACTIVELOOP_ORG')

embeddings = OpenAIEmbeddings()

dataset_path = 'hub://' + org + '/data'
print(dataset_path)

hub://will-nova/data


In [16]:
my_contract = "../declaration_of_independence.txt"

In [18]:
with open(my_contract) as f:
    contract = f.read()
# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=100)
pages = text_splitter.split_text(contract)
texts = text_splitter.create_documents(pages)

print (texts)

[Document(page_content='In Congress, July 4, 1776', metadata={}), Document(page_content='The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the', metadata={}), Document(page_content="the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires", metadata={}), Document(page_content="of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.", metadata={}), Document(page_content='We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain

In [19]:
# dataset_path = 'hub://'+org+'/data'
embeddings = OpenAIEmbeddings()
db = DeepLake.from_documents(texts, embeddings, dataset_path=dataset_path, overwrite=True)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/will-nova/data
hub://will-nova/data loaded successfully.


Evaluating ingest: 100%|██████████████████████████████████████| 1/1 [00:10<00:00
-

Dataset(path='hub://will-nova/data', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (51, 1536)  float32   None   
    ids      text     (51, 1)      str     None   
 metadata    json     (51, 1)      str     None   
   text      text     (51, 1)      str     None   


In [20]:
db = DeepLake(dataset_path=dataset_path, read_only=True, embedding_function=embeddings)

retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/will-nova/data



hub://will-nova/data loaded successfully.



Deep Lake Dataset in hub://will-nova/data already exists, loading from the storage
Dataset(path='hub://will-nova/data', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (51, 1536)  float32   None   
    ids      text     (51, 1)      str     None   
 metadata    json     (51, 1)      str     None   
   text      text     (51, 1)      str     None   


In [21]:
query = input("Enter query:")
ans = qa({"query": query})

print("\nCustom-ChatGPT-app answer: " + ans['result'] + "\n\n")
print(ans)


Enter query: when was this written?



Custom-ChatGPT-app answer:  July 4, 1776


{'query': 'when was this written?', 'result': ' July 4, 1776', 'source_documents': [Document(page_content='In Congress, July 4, 1776', metadata={}), Document(page_content='The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the', metadata={}), Document(page_content='of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation.', metadata={}), Document(page_content='may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor.', metadata={})]}


In [22]:
query = input("Enter query:")
ans = qa({"query": query})

print("\nCustom-ChatGPT-app answer: " + ans['result'] + "\n\n")
print(ans)


Enter query: why do we need to die?



Custom-ChatGPT-app answer:  We don't need to die. The context is talking about the necessity of separating from the British and holding them as enemies in war, but not necessarily dying.


{'query': 'why do we need to die?', 'result': " We don't need to die. The context is talking about the necessity of separating from the British and holding them as enemies in war, but not necessarily dying.", 'source_documents': [Document(page_content='necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.', metadata={}), Document(page_content='may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor.', metadata={}), Document(page_content='We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among t

In [23]:
query = input("Enter query:")
ans = qa({"query": query})

print("\nCustom-ChatGPT-app answer: " + ans['result'] + "\n\n")
print(ans)


Enter query: what's the new foreign policy?



Custom-ChatGPT-app answer:  The new foreign policy is to encourage the migration of foreigners to these states and to pass laws to support them.


{'query': "what's the new foreign policy?", 'result': ' The new foreign policy is to encourage the migration of foreigners to these states and to pass laws to support them.', 'source_documents': [Document(page_content='others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands.', metadata={}), Document(page_content='He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of', metadata={}), Document(page_content='He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destru

In [27]:
query = input("Enter query:")
ans = qa({"query": query})

print("\nCustom-ChatGPT-app answer: " + ans['result'] + "\n\n")
print(ans)


Enter query: create test flashcards for a 1st grader



Custom-ChatGPT-app answer: 

Flashcard 1: What is a Tyranny?
Answer: A tyranny is an oppressive form of government.

Flashcard 2: What is an Arbitrary government?
Answer: An arbitrary government is a form of government in which a ruler has unlimited power.


{'query': 'create test flashcards for a 1st grader', 'result': '\n\nFlashcard 1: What is a Tyranny?\nAnswer: A tyranny is an oppressive form of government.\n\nFlashcard 2: What is an Arbitrary government?\nAnswer: An arbitrary government is a form of government in which a ruler has unlimited power.', 'source_documents': [Document(page_content='so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies:', metadata={}), Document(page_content='having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.', metadata={}), Document(page_content='For abolishing the free System of English Laws in a neighb

In [25]:
query = input("Enter query:")
ans = qa({"query": query})

print("\nCustom-ChatGPT-app answer: " + ans['result'] + "\n\n")
print(ans)


Enter query: create more test flashcards for a 7th grader.



Custom-ChatGPT-app answer:  Make flashcards about the Declaration of Independence. For example, one flashcard could ask "What did the Declaration of Independence say about taxation without representation?" and the answer on the back of the card could say "The Declaration of Independence declared that it was wrong for the British government to impose taxes on the colonies without giving them representation in Parliament."


{'query': 'create more test flashcards for a 7th grader.', 'result': ' Make flashcards about the Declaration of Independence. For example, one flashcard could ask "What did the Declaration of Independence say about taxation without representation?" and the answer on the back of the card could say "The Declaration of Independence declared that it was wrong for the British government to impose taxes on the colonies without giving them representation in Parliament."', 'source_documents': [Document(page_content='so as to render it at once an example and fit instrument f